# 外皮情報LV1からLV2へのコンバート

## 1. 外皮簡易法＋用途別床面積
- 当該住戸の外皮の部位の面積等を用いずに簡易熱負荷計算を実施する場合には、部位の面積および長さを、「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）」２．エネルギー消費性能の算定方法　2.2　算定方法　第三章　 暖冷房負荷と外皮性能　	第二節　外皮性能　9.当該住戸の外皮の部位の面積等を用いずに外皮性能を評価する方法　に示された標準住戸における部位の面積および長さ等より推定する。
- 屋根・天井、外壁、床の構成は、天井入力値である部位種別の熱貫流率と一致するよう、木造の標準的な構成における断熱材の厚さを0mm以上の範囲で調整する。木造の標準的な構成は、SimHeatでの熱負荷計算用の層構成決定に用いたQ値計算における、旧Ⅳ地域の平成11年基準の層構成とする。
- 土間床等の外周部の線熱貫流率については、簡易熱負荷計算での扱いに従う
- 開口部の日射熱取得率および外部日除けの取得日射熱補正係数は、それぞれ、季節別に、垂直面日射熱取得率および取得日射熱補正係数の入力値もしくは規定値を使用する。この場合、簡易熱負荷計算においては、開口部の入射角特性および外部日除けは考慮しない。
- その他は2.と同様に求める。

#### 《設計住戸の各部位の長さおよび面積の推定》
$$ \qquad
A_{i,d}= A_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (1)
\\
$$
$$ \qquad
L_{i,d}= L_{i,s}・\frac{A_{floor,d}}{A_{floor,s}}
\qquad (2)
\\
$$

- 入力値：

  - 標準住戸の延床面積$[m ^ 2]$：$A_{floor,s}$
  - 設計住戸の延床面積$[m ^ 2]$：$A_{floor,d}$
  - 標準住戸の部位（一般部位又は開口部）𝑖の面積$[m ^ 2]$：$A_{i,s}$
  - 標準住戸の部位（土間床等の外周部）𝑖の長さ$[m]$：$L_{i,s}$

- 出力値：

  - 設計住戸の部位（一般部位又は開口部）𝑖の面積の推定値$[m ^ 2]$：$A_{i,d}$
  - 設計住戸の部位（土間床等の外周部）𝑖の長さの推定値$[m]$：$L_{i,d}$

- 屋根・天井壁体構成

| 材料 | 厚さ[m] | 熱伝導率[W/(m・K)] | 熱抵抗[（㎡K・W）] | 容積比熱[J/（L・K）] |
|-------------|-------------|-------------|-------------|
| Ro | - | - | 0.090 | 0 |
| 住宅用グラスウール断熱材10K相当 | <font color="red">0.2</font> | 0.05 | <font color="red">4.000</font> | 8 |
| せっこうボード | 0.0095 | 0.22 | 0.043 | 830 |
| Ri | - | - | 0.090 | 0 |

- 外壁体構成

| 材料 | 厚さ[m] | 熱伝導率[W/(m・K)] | 熱抵抗[（㎡K・W）] | 容積比熱[J/（L・K）] |
|-------------|-------------|-------------|-------------|
| Ro | - | - | 0.110 | 0 |
| 合板 | 0.012 | 0.16 | 0.075 | 720 |
| 住宅用グラスウール断熱材16K相当 | <font color="red">0.099</font> | 0.045 | <font color="red">2.200</font> | 13 |
| 密閉空気層 | - | - | 0.090 | 0 |
| せっこうボード | 0.0095 | 0.22 | 0.043 | 830 |
| Ri | - | - | 0.110 | 0 |

- 床壁体構成

| 材料 | 厚さ[m] | 熱伝導率[W/(m・K)] | 熱抵抗[（㎡K・W）] | 容積比熱[J/（L・K）] |
|-------------|-------------|-------------|-------------|
| Ro | - | - | 0.15 | - | - |
| 住宅用グラスウール断熱材16K相当 | <font color="red">0.099</font> | 0.045 | <font color="red">2.200</font> | 13 |
| 合板 | 0.012 | 0.16 | 0.075 | 720 |
| Ri | - | - | 0.15 | - | - |

#### コンバート【分類：共通】

- レベル1：
  - 地域区分$[-]$：Region
  - 主たる居室の床面積$[m^2]$：MainOccupantRoomFloorArea
  - その他の居室の床面積$[m^2]$：OtherOccupantRoomFloowArea
  - 非居室の床面積$[m^2]$：NonOccupantRoomFloorArea
   
- レベル2：
  - 地域区分$[-]$：Region
  - 標準住戸フラグ$[-]$：FlagStandardHouse
  - 主たる居室の床面積$[m^2]$：MainOccupantRoomFloorArea
  - その他の居室の床面積$[m^2]$：OtherOccupantRoomFloowArea
  - 非居室の床面積$[m^2]$：NonOccupantRoomFloorArea

In [1]:
data1_common = {
    'Region' : 6,
    'MainOccupantRoomFloorArea': 30.0,
    'OtherOccupantRoomFloowArea': 60.0,
    'NonOccupantRoomFloorArea': 30.0,
}

data1_common

{'MainOccupantRoomFloorArea': 30.0,
 'NonOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloowArea': 60.0,
 'Region': 6}

In [2]:
def convert_1to2_common(d1):
    return {
        'Region'                     : d1['Region'],
        'FlagStandardHouse'          : True,
        'MainOccupantRoomFloorArea'  : d1['MainOccupantRoomFloorArea'],
        'OtherOccupantRoomFloowArea' : d1['OtherOccupantRoomFloowArea'],
        'NonOccupantRoomFloorArea'   : d1['NonOccupantRoomFloorArea']
           }

In [3]:
data2_common = convert_1to2_common(data1_common)
data2_common

{'FlagStandardHouse': True,
 'MainOccupantRoomFloorArea': 30.0,
 'NonOccupantRoomFloorArea': 30.0,
 'OtherOccupantRoomFloowArea': 60.0,
 'Region': 6}

#### コンバート【分類：壁体構成】

- レベル1：
  - 壁体名称$[-]$：$name$
  - 部位種別$[-]$：$type$
  - 平均熱貫流率$[W/(m^2・K)]$：$U_a$
- レベル2：
  - 壁体名称$[-]$：$name$
  - 部位種別$[-]$：$type$
  - 部材名称$[-]$：$matrial$
  - 厚さ$[m]$：$d$
  - 熱伝達率$[W/(m・K)]$：$λ$
  - 容積比熱$[J/(m^3・K)]$：$Cρ$
  - 平均熱貫流率$[W/(m^2・K)]$：$U_a$
  - 一般部熱貫流率$[W/(m^2・K)]$：$U_g$
  - 一般部構造種別$[-]$：$Structure$
  - 木造熱橋部計算方法$[-]$：$HeatBridge$
  - 熱橋部熱面積比率$[-]$：$a_{hb}$
  - 熱橋部熱貫流率$[W/(m^2・K)]$：$U_{hb}$
  - 熱橋長さ$[m]$：$L_i$
  - 線熱貫流率$[W/(m・K)]$：$Ψ_i$
  - 補正熱貫流率$[W/(m^2・K)]$：$U_r$

In [4]:
data1_wall = {
    'name': 'Sample Wall',
    'type': 2,
    'Ua': 0.538
}

data1_wall

{'Ua': 0.538, 'name': 'Sample Wall', 'type': 2}

In [5]:
import copy

def convert_1to2_wall(_data1_wall):
    data_wall = copy.deepcopy(_data1_wall)
    
    for key in data_wall.keys():
        data_wall[key] = data_wall[key]
 
    for i in ['Ug','Structure','HeatBridge','ahb','Uhb','Ur']:
        data_wall[i] = None
        
    data_wall['HeatBridges']= {'HeatBridge1':{'L':None,'Ψ':None}, 
                               'HeatBridge2':{'L':None,'Ψ':None},
                               'HeatBridge3':{'L':None,'Ψ':None}}
    if data_wall['type']==1:
        d_insulation=round((1/data_wall['Ua']-(0.090+0.0095/0.22+0.090))*0.05,3)
        data_wall['Layers']= {'Layer1':{'material':'GW10K','d':d_insulation,'λ':0.05,'Cρ':8}, 
                          'Layer2':{'material':'GPB','d':0.0095,'λ':0.22,'Cρ':830}}    
    elif data_wall['type']==2:
        d_insulation=round((1/data_wall['Ua']-(0.11+0.012/0.16+0.09+0.0095/0.22+0.11))*0.045,3)
        data_wall['Layers']= {'Layer1':{'material':'PED','d':0.012,'λ':0.16,'Cρ':720}, 
                          'Layer2':{'material':'GW16K','d':d_insulation,'λ':0.045,'Cρ':13},
                          'Layer3':{'material':'Air','d':1,'λ':round(1/0.090,2),'Cρ':0},
                          'Layer4':{'material':'GPB','d':0.0095,'λ':0.22,'Cρ':830}}
    elif data_wall['type']==3:
        d_insulation=round((1/data_wall['Ua']-(0.15+0.012/0.16+0.15))*0.045,3)
        data_wall['Layers']= {'Layer1':{'material':'PED','d':0.012,'λ':0.16,'Cρ':720}, 
                          'Layer2':{'material':'GW16K','d':d_insulation,'λ':0.045,'Cρ':13}}
    return data_wall

In [6]:
data2_wall=convert_1to2_wall(data1_wall)
data2_wall

{'HeatBridge': None,
 'HeatBridges': {'HeatBridge1': {'L': None, 'Ψ': None},
  'HeatBridge2': {'L': None, 'Ψ': None},
  'HeatBridge3': {'L': None, 'Ψ': None}},
 'Layers': {'Layer1': {'Cρ': 720, 'd': 0.012, 'material': 'PED', 'λ': 0.16},
  'Layer2': {'Cρ': 13, 'd': 0.064, 'material': 'GW16K', 'λ': 0.045},
  'Layer3': {'Cρ': 0, 'd': 1, 'material': 'Air', 'λ': 11.11},
  'Layer4': {'Cρ': 830, 'd': 0.0095, 'material': 'GPB', 'λ': 0.22}},
 'Structure': None,
 'Ua': 0.538,
 'Ug': None,
 'Uhb': None,
 'Ur': None,
 'ahb': None,
 'name': 'Sample Wall',
 'type': 2}

#### コンバート【分類：開口部】

- レベル1：
  - 開口部名称$[-]$：$name$
  - 日射熱取得率（夏期）$[-]$：$η_c$
  - 日射熱取得率（冬期）$[-]$：$η_h$
  - 熱貫流率$[W/(m^2・K)]$：$U$ 
- レベル2：
  - 開口部名称$[-]$：$name$
  - 日射熱取得率$[-]$：$η$
  - 日射熱取得率（夏期）$[-]$：$η_c$
  - 日射熱取得率（冬期）$[-]$：$η_h$
  - 熱貫流率$[W/(m^2・K)]$：$U$ 

In [7]:
data1_window = {
    'name': 'Sample Window',
    'ηc': 0.738,
    'ηh': 0.738,
    'U': 4.65
}

data1_window

{'U': 4.65, 'name': 'Sample Window', 'ηc': 0.738, 'ηh': 0.738}

In [8]:
import copy

def convert_1to2_window(_data1_window):
    data_window = copy.deepcopy(_data1_window)
    
    for key in data_window.keys():
        data_window[key] = data_window[key]
    
    data_window['η'] = None
    
    return data_window

In [9]:
data2_window=convert_1to2_window(data1_window)
data2_window

{'U': 4.65, 'name': 'Sample Window', 'η': None, 'ηc': 0.738, 'ηh': 0.738}

#### コンバート【分類：外部日除け】

- レベル1：
  - 外部日除け名称$[-]$：$name$
  - 取得日射熱補正係数（夏期）$[-]$：$f_c$
  - 取得日射熱補正係数（冬期）$[-]$：$f_h$
- レベル2：
  - 外部日除け名称$[-]$：$name$
  - 開口部高さ$[m]$：$y_2$
  - 出巾$[m]$：$z$
  - 窓上端と外部日除けの距離$[m]$：$y_1$
  - 取得日射熱補正係数（夏期）$[-]$：$f_c$
  - 取得日射熱補正係数（冬期）$[-]$：$f_h$

In [10]:
data1_sunshade = {
    'name': 'Sample Sunshade',
    'fc': 0.700,
    'fh': 0.600,
}

data1_sunshade

{'fc': 0.7, 'fh': 0.6, 'name': 'Sample Sunshade'}

In [11]:
import copy

def convert_1to2_sunshade(_data1_sunshade):
    data_sunshade = copy.deepcopy(_data1_sunshade)
    
    for key in data_sunshade.keys():
        data_sunshade[key] = data_sunshade[key]

    for i in ['y1','y2','z']:
        data_sunshade[i] = None
        
    return data_sunshade

In [12]:
data2_sunshade=convert_1to2_sunshade(data1_sunshade)
data2_sunshade

{'fc': 0.7,
 'fh': 0.6,
 'name': 'Sample Sunshade',
 'y1': None,
 'y2': None,
 'z': None}

#### コンバート【分類：部位情報】

- レベル1：
  - 外皮フラグ$[-]$：$FlagPerimeter$
  - 日射フラグ$[-]$：$FlagSolarRadiation$
  - 方位$[-]$：$direction$
  - 非定常フラグ$[-]$：$FlagUnsteady$
  - 壁体・開口部名称$[-]$：$name$
  - 部位面積$[m^2]$：$A$
  - 外部日除け名称$[-]$：$sunshade$
- レベル2：
  - 外皮フラグ$[-]$：$FlagPerimeter$
  - 日射フラグ$[-]$：$FlagSolarRadiation$
  - 方位$[-]$：$direction$
  - 非定常フラグ$[-]$：$FlagUnsteady$
  - 壁体・開口部名称$[-]$：$name$
  - 部位面積$[m^2]$：$A$
  - 外部日除け名称$[-]$：$sunshade$

In [13]:
data1_element = {
    'name': 'Sample Element',
    'FlagPerimeter': 1,
    'FlagSolarRadiation': 1,
    'FlagUnsteady': 1,
    'direction': 1,
    'A': 1,
    'Sunshade': None
}

data1_element

{'A': 1,
 'FlagPerimeter': 1,
 'FlagSolarRadiation': 1,
 'FlagUnsteady': 1,
 'Sunshade': None,
 'direction': 1,
 'name': 'Sample Element'}

In [14]:
import copy

def convert_1to2_element(_data1_element,data1_common):
    data_element = copy.deepcopy(_data1_element)
    
    for key in data_element.keys():
        data_element[key] = data_element[key]
    
    
    Afloor=sum([data1_common['Afloor1'],data1_common['Afloor2'],data1_common['Afloor3']])
    data_element['A'] = round(data_element['A']*Afloor/90,2)
        
    return data_element

In [15]:
data2_element=convert_1to2_element(data1_element,data1_common)
data2_element

KeyError: 'Afloor1'

In [ ]:
#標準住戸における部位の面積及び土間床等の外周部の長さ等
#（い）床断熱住戸の場合
#番号,外皮フラグ[-],日射フラグ[-],方位[-],非定常フラグ[-],壁体・開口部名称[-],部位面積[㎡],外部日除け名称[-]
#ElementNo,FlagPerimeter,FlagSolarRadiation,direction,FlagUnsteady,name,A,Sunshade

import os,csv
CsvFile = csv.reader(open('（い）床断熱住戸の場合.csv'),delimiter=',')

StandardHouseElemntsFI = []
for i in CsvFile:
    StandardHouseElemntsFI.append(i)

def get_StandardHouseElemntsFI(StandardHouseElemntsFI,i):
    data_elements = {'name': StandardHouseElemntsFI[i][5],
                     'FlagPerimeter': int(StandardHouseElemntsFI[i][1]),
                     'FlagSolarRadiation': int(StandardHouseElemntsFI[i][2]),
                     'FlagUnsteady': int(StandardHouseElemntsFI[i][4]),
                     'direction': int(StandardHouseElemntsFI[i][3]),
                     'A': float(StandardHouseElemntsFI[i][6]),
                     'Sunshade': StandardHouseElemntsFI[i][7]}        
    return data_elements

def make_data_elements_list(data2_element,i):
    data_elements_list = [i,int(data2_element['FlagPerimeter']),int(data2_element['FlagSolarRadiation']),
                          int(data2_element['direction']),int(data2_element['FlagUnsteady']),
                          data2_element['name'],float(data2_element['A']),data2_element['Sunshade']]   
    return data_elements_list

data1_element_list = [['ElementNo','FlagPerimeter','FlagSolarRadiation',
                       'direction','FlagUnsteady','name','A','Sunshade']]
for i in range(1,len(StandardHouseElemntsFI)):
    data1_element=get_StandardHouseElemntsFI(StandardHouseElemntsFI,i)
    data2_element=convert_1to2_element(data1_element,data1_common)
    #print("data1_element{0}={1}".format(i,data2_element))
    data1_element_list.append(make_data_elements_list(data2_element,i))

import csv
with open('（い）床断熱住戸の場合_補完後.csv', 'w') as f:
    writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
    writer.writerows(data1_element_list) # 2次元配列も書き込める

In [ ]:
#標準住戸における部位の面積及び土間床等の外周部の長さ等
#（ろ）基礎断熱住戸の場合
#番号,外皮フラグ[-],日射フラグ[-],方位[-],非定常フラグ[-],壁体・開口部名称[-],部位面積[㎡],外部日除け名称[-]
#ElementNo,FlagPerimeter,FlagSolarRadiation,direction,FlagUnsteady,name,A,Sunshade

import os,csv
CsvFile = csv.reader(open('（ろ）基礎断熱住戸の場合.csv'),delimiter=',')

StandardHouseElemntsFI = []
for i in CsvFile:
    StandardHouseElemntsFI.append(i)

def get_StandardHouseElemntsFI(StandardHouseElemntsFI,i):
    data_elements = {'name': StandardHouseElemntsFI[i][5],
                     'FlagPerimeter': int(StandardHouseElemntsFI[i][1]),
                     'FlagSolarRadiation': int(StandardHouseElemntsFI[i][2]),
                     'FlagUnsteady': int(StandardHouseElemntsFI[i][4]),
                     'direction': int(StandardHouseElemntsFI[i][3]),
                     'A': float(StandardHouseElemntsFI[i][6]),
                     'Sunshade': StandardHouseElemntsFI[i][7]}        
    return data_elements

def make_data_elements_list(data2_element,i):
    data_elements_list = [i,int(data2_element['FlagPerimeter']),int(data2_element['FlagSolarRadiation']),
                          int(data2_element['direction']),int(data2_element['FlagUnsteady']),
                          data2_element['name'],float(data2_element['A']),data2_element['Sunshade']]   
    return data_elements_list

data1_element_list = [['ElementNo','FlagPerimeter','FlagSolarRadiation',
                       'direction','FlagUnsteady','name','A','Sunshade']]
for i in range(1,len(StandardHouseElemntsFI)):
    data1_element=get_StandardHouseElemntsFI(StandardHouseElemntsFI,i)
    data2_element=convert_1to2_element(data1_element,data1_common)
    #print("data1_element{0}={1}".format(i,data2_element))
    data1_element_list.append(make_data_elements_list(data2_element,i))

import csv
with open('（ろ）基礎断熱住戸の場合_補完後.csv', 'w') as f:
    writer = csv.writer(f, lineterminator='\n') # 改行コード（\n）を指定しておく
    writer.writerows(data1_element_list) # 2次元配列も書き込める

In [ ]:
#標準住戸における部位の面積及び土間床等の外周部の長さ等
#（い）床断熱住戸の場合
#番号,外皮フラグ[-],日射フラグ[-],方位[-],非定常フラグ[-],壁体・開口部名称[-],部位面積[㎡],外部日除け名称[-]
#ElementNo,FlagPerimeter,FlagSolarRadiation,direction,FlagUnsteady,name,A,Sunshade

import numpy as np
StandardHouseElemntsFI = np.array([[1,1,1,1,1,'Aroof',50.85,None],[2,1,1,7,1,'AwallSW',30.47,None],
    [3,1,1,9,1,'AwallNW',22.37,None],[4,1,1,3,1,'AwallNE',47.92,None],[5,1,1,5,1,'AwallSE',22.28,None],
    [6,1,1,7,2,'AdoorSW',0,None],[7,1,1,9,2,'AdoorNW',1.89,None],[8,1,1,3,2,'AdoorNE',1.62,None],
    [9,1,1,5,2,'AdoorSE',0,None],[10,1,1,7,2,'AwndSW',22.69,'SunshadeSW'],[11,1,1,9,2,'AwndNW',2.38,'SunshadeNW'],
    [12,1,1,3,2,'AwndNE',3.63,'SunshadeNE'],[13,1,1,5,2,'AwndSE',4.37,'SunshadeSE'],
    [14,1,2,12,1,'AIF',45.05,None],[15,1,1,7,1,'AbaseSW',0,None],[16,1,1,9,1,'AbaseNW',0.91,None],
    [17,1,1,3,1,'AbaseNE',0.91,None],[18,1,1,5,1,'AbaseSE',0,None],[19,1,2,12,1,'AbaseIF',1.82,None],
    [20,1,1,7,1,'AbasedSW',0,None],[21,1,1,9,1,'AbasedNW',0.33,None],[22,1,1,3,1,'AbasedNE',0.25,None],
    [23,1,1,5,1,'AbasedSE',0,None],[24,1,2,12,1,'AbasedIF',0.57,None],[25,1,1,7,1,'LprmSW',0,None],
    [26,1,1,9,1,'LprmNW',1.82,None],[27,1,1,3,1,'LprmNE',1.82,None],[28,1,1,5,1,'LprmSE',0,None],
    [29,1,2,12,1,'LprmIF',3.64,None],[30,1,1,7,1,'LprmdSW',0,None],[31,1,1,9,1,'LprmdNW',1.82,None],
    [32,1,1,3,1,'LprmdNE',1.37,None],[33,1,1,5,1,'prmdSE',0,None],[34,1,2,12,1,'LprmdIF',3.19,None]])

def get_StandardHouseElemntsFI(StandardHouseElemntsFI,i):
    data_elements = {'name': StandardHouseElemntsFI[i,5],
                     'FlagPerimeter': StandardHouseElemntsFI[i,1],
                     'FlagSolarRadiation': StandardHouseElemntsFI[i,2],
                     'FlagUnsteady': StandardHouseElemntsFI[i,4],
                     'direction': StandardHouseElemntsFI[i,3],
                     'A': StandardHouseElemntsFI[i,6],
                     'Sunshade': StandardHouseElemntsFI[i,7]}        
    return data_elements
    
for i in range(0,len(StandardHouseElemntsFI)):
    data1_element=get_StandardHouseElemntsFI(StandardHouseElemntsFI,i)
    #print("data1_element{0}={1}".format(i,data1_element))

In [ ]:
#標準住戸における部位の面積及び土間床等の外周部の長さ等
#（ろ）基礎断熱住戸の場合 
#番号,外皮フラグ[-],日射フラグ[-],方位[-],非定常フラグ[-],壁体・開口部名称[-],部位面積[㎡],外部日除け名称[-]
#ElementNo,FlagPerimeter,FlagSolarRadiation,direction,FlagUnsteady,name,A,Sunshade

import numpy as np
StandardHouseElemntsBI = np.array([[1,1,1,1,1,'Aroof',50.85,None],[2,1,1,7,1,'AwallSW',30.47,None],
    [3,1,1,9,1,'AwallNW',22.37,None],[4,1,1,3,1,'AwallNE',47.92,None],[5,1,1,5,1,'AwallSE',22.28,None],
    [6,1,1,7,2,'AdoorSW',0,None],[7,1,1,9,2,'AdoorNW',1.89,None],[8,1,1,3,2,'AdoorNE',1.62,None],
    [9,1,1,5,2,'AdoorSE',0,None],[10,1,1,7,2,'AwndSW',22.69,'SunshadeSW'],[11,1,1,9,2,'AwndNW',2.38,'SunshadeNW'],
    [12,1,1,3,2,'AwndNE',3.63,'SunshadeNE'],[13,1,1,5,2,'AwndSE',4.37,'SunshadeSE'],[14,1,2,12,1,'AIF',0,None],
    [15,1,1,7,1,'AbaseSW',5.3,None],[16,1,1,9,1,'AbaseNW',1.48,None],[17,1,1,3,1,'AbaseNE',4.62,None],
    [18,1,1,5,1,'AbaseSE',2.4,None],[19,1,2,12,1,'AbaseIF',0,None],[20,1,1,7,1,'AbasedSW',0,None],
    [21,1,1,9,1,'AbasedNW',0.33,None],[22,1,1,3,1,'AbasedNE',0.25,None],[23,1,1,5,1,'AbasedSE',0,None],
    [24,1,2,12,1,'AbasedIF',0,None],[25,1,1,7,1,'LprmSW',10.61,None],[26,1,1,9,1,'LprmNW',2.97,None],
    [27,1,1,3,1,'LprmNE',9.24,None],[28,1,1,5,1,'LprmSE',4.79,None],[29,1,2,12,1,'LprmIF',0,None],
    [30,1,1,7,1,'LprmdSW',0,None],[31,1,1,9,1,'LprmdNW',1.82,None],[32,1,1,3,1,'LprmdNE',1.37,None],
    [33,1,1,5,1,'LprmdSE',0,None],[34,1,2,12,1,'LprmdIF',0,None]])

def get_StandardHouseElemntsBI(StandardHouseElemntsBI,i):
    data_elements = {'name': StandardHouseElemntsBI[i,5],
                     'FlagPerimeter': StandardHouseElemntsBI[i,1],
                     'FlagSolarRadiation': StandardHouseElemntsBI[i,2],
                     'FlagUnsteady': StandardHouseElemntsBI[i,4],
                     'direction': StandardHouseElemntsBI[i,3],
                     'A': StandardHouseElemntsBI[i,6],
                     'Sunshade': StandardHouseElemntsBI[i,7]}        
    return data_elements
    
for i in range(0,len(StandardHouseElemntsBI)):
    data1_element=get_StandardHouseElemntsBI(StandardHouseElemntsBI,i)
    #print("data1_element{0}={1}".format(i,data1_element))